In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pandas as pd

Using TensorFlow backend.


In [2]:
import os
os.listdir('/kaggle/input/')

['nlp-test']

In [3]:
data = pd.read_excel("/kaggle/input/nlp-test/Human_Organisation_Bios.xlsx")

In [4]:
y = data['Classification'].apply(lambda x: 1 if x=='Organisation' else 0)
Y = y.values.reshape(-1,1)
X_train=data['Bios'].astype(str)
Y_train=Y
max_words = 5000
max_len = 1024
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [5]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=100,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
model.save('LSTM_text.h5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1024)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1024, 50)          250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [6]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
ypred = model.predict(test_sequences_matrix)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

NameError: name 'X_test' is not defined

In [7]:
# test_sequences = tok.texts_to_sequences(['Creative available for commission work. Specializing in concept development, illustration, murals, graphics and 3-D set design & construction.'])
# test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
# ypred = model.predict(test_sequences_matrix)
# print(ypred[0][0])